In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import datetime as dtm
import matplotlib.dates as mpd
import pytz
tzutc = pytz.timezone('UTC')

#import operator
import math
import random
import numpy
import scipy
import scipy.optimize as spo
from scipy import interpolate
import scipy.constants
import itertools
import sys
#import scipy.optimize as spo
import os
import operator
#from PIL import Image as ipp
import multiprocessing as mpp
#
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import json
import pickle
#

import geopy.distance
#from geopy.distance import vincenty
#from geopy.distance import great_circle
#
#import shapely.geometry as sgp
os.environ['PROJ_LIB'] = '{}/anaconda3/share/proj'.format(os.getenv('HOME'))
#
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from geographiclib.geodesic import Geodesic as ggp
#

#import ANSStools as atp
from yodiipy import ANSStools as atp
#
import contours2kml
import globalETAS as gep

#import global_etas_auto as ggep

from eq_params import *
#
from nepal_figs import *
import optimizers
#
import random
import geopy

In [7]:
# let's just get a catalog and pull an earthquake out of it.
n_cpu=None
#
lat0 = 35.705
lon0 = -117.506
#
#ll_sacramento = (lon0, lat0)

#m0 = 7.8

d_lat=2.
d_lon=2.
#
lats = [lat0-d_lat, lat0+d_lat]
lons = [lon0-d_lon, lon0+d_lon]
#
to_dt = dtm.datetime(2019,7,6, tzinfo=tzutc)
eq_prams = {'do_recarray': True, 'D_fract': 1.5,
               't_0':dtm.datetime(1990, 1, 1, 0, 0, 0, tzinfo=tzutc),
               't_now':to_dt, 't_future':None ,
               'lats': lats, 'p_cat': 1.1, 'b1': 1.0, 'mc': 2.5, 'q_cat': 1.5,
               'p_etas':1.1, 'q_etas':1.5,
               'lons': lons, 'dmstar': 1.0, 'b2': 1.5, 'd_tau': 2.28,
               'incat': None, 'fit_factor': 2.0, 'd_lambda': 1.76, 'etas_range_padding':1.5,
            'etas_range_factor':30.0, 'ab_ratio_expon':.25 }
eq_prams['mc'] = 6.7
eq_prams['t0'] = dtm.datetime(2019,6,20, 0,0,0,0, tzinfo=pytz.timezone('UTC'))
#

mycat = atp.cat_from_anss_comcat(lon=lons, lat=lats, minMag=2.5,
                        dates0=[dtm.datetime(2005,1,1, tzinfo=pytz.timezone('UTC')), 
                                dtm.datetime.now(pytz.timezone('UTC'))],
                            Nmax=None, fout=None, rec_array=True)
#                        dates0=[dtm.datetime(2005,1,1, tzinfo=tzutc), None], Nmax=None, fout=None, rec_array=True)

mycat = gep.make_ETAS_catalog_mpp(incat=mycat, n_cpu=n_cpu)
#

etas_prams:  {'incat': None, 'lats': [32.0, 38.0], 'lons': [-117.0, -114.0], 'mc': 2.5, 'date_range': ['1990-1-1', None], 'D_fract': 1.5, 'd_lambda': 1.76, 'd_tau': 2.28, 'fit_factor': 1.5, 'p': 1.1, 'q': 1.5, 'dmstar': 1.0, 'b1': 1.0, 'b2': 1.5, 'do_recarray': False}
results fetched.


In [8]:
for rw in mycat:
    if rw['mag']>7.0:
        eq_ms = rw
    #
#
print('mainshock: ', eq_ms)

mainshock:  ('2019-07-06T03:19:53.040000', 35.7695, -117.5993333, 7.1, 8., 737246.13880833, 61.65950019, 1277.03847158, 1.40581446e-05, 18.62087137, 165.68091867, 0.00150686, 1., 1.1, 1.5, 26.80175629, [0.09050061, 0.05666743], [[ 0.82796634,  0.56077779], [-0.56077779,  0.82796634]], 3225.)


In [9]:
eq = gep.Earthquake(eq_ms)

In [10]:
print(eq.local_intensity(lon=-117., lat=35.) )
print(eq.local_intensity(lon=-117., lat=34.) )

1.0225408109874048e-12
8.312145139055901e-13


In [11]:
eq.t_0, eq.t_1, eq.t_2

(165.68091866556017, 737340.6393351462, None)

In [12]:
ts = numpy.linspace(eq.t_1, eq.t_1+100, 100)

#ts[20] = None

In [13]:
#orates = eq.omori_rate(t=ts)
# this works, but eq.orate() does not; eq.orate() fails on some sort of validation (.any(), .all() reference)
orates = 1./(eq.tau*(eq.t_0 + ts))

print('** orates: ', orates)

** orates:  [0.00089983 0.00089983 0.00089983 0.00089983 0.00089983 0.00089983
 0.00089982 0.00089982 0.00089982 0.00089982 0.00089982 0.00089982
 0.00089982 0.00089982 0.00089981 0.00089981 0.00089981 0.00089981
 0.00089981 0.00089981 0.00089981 0.00089981 0.00089981 0.0008998
 0.0008998  0.0008998  0.0008998  0.0008998  0.0008998  0.0008998
 0.0008998  0.00089979 0.00089979 0.00089979 0.00089979 0.00089979
 0.00089979 0.00089979 0.00089979 0.00089978 0.00089978 0.00089978
 0.00089978 0.00089978 0.00089978 0.00089978 0.00089978 0.00089977
 0.00089977 0.00089977 0.00089977 0.00089977 0.00089977 0.00089977
 0.00089977 0.00089976 0.00089976 0.00089976 0.00089976 0.00089976
 0.00089976 0.00089976 0.00089976 0.00089975 0.00089975 0.00089975
 0.00089975 0.00089975 0.00089975 0.00089975 0.00089975 0.00089974
 0.00089974 0.00089974 0.00089974 0.00089974 0.00089974 0.00089974
 0.00089974 0.00089973 0.00089973 0.00089973 0.00089973 0.00089973
 0.00089973 0.00089973 0.00089973 0.00089972 0.00089

In [14]:
print('** orates: ', orates)

** orates:  [0.00089983 0.00089983 0.00089983 0.00089983 0.00089983 0.00089983
 0.00089982 0.00089982 0.00089982 0.00089982 0.00089982 0.00089982
 0.00089982 0.00089982 0.00089981 0.00089981 0.00089981 0.00089981
 0.00089981 0.00089981 0.00089981 0.00089981 0.00089981 0.0008998
 0.0008998  0.0008998  0.0008998  0.0008998  0.0008998  0.0008998
 0.0008998  0.00089979 0.00089979 0.00089979 0.00089979 0.00089979
 0.00089979 0.00089979 0.00089979 0.00089978 0.00089978 0.00089978
 0.00089978 0.00089978 0.00089978 0.00089978 0.00089978 0.00089977
 0.00089977 0.00089977 0.00089977 0.00089977 0.00089977 0.00089977
 0.00089977 0.00089976 0.00089976 0.00089976 0.00089976 0.00089976
 0.00089976 0.00089976 0.00089976 0.00089975 0.00089975 0.00089975
 0.00089975 0.00089975 0.00089975 0.00089975 0.00089975 0.00089974
 0.00089974 0.00089974 0.00089974 0.00089974 0.00089974 0.00089974
 0.00089974 0.00089973 0.00089973 0.00089973 0.00089973 0.00089973
 0.00089973 0.00089973 0.00089973 0.00089972 0.00089

In [15]:
#print('** evecs, evals: ', eq.e_vals, eq.e_vecs)

In [16]:
lons_lats = numpy.array([[lon, lat] for lon,lat in itertools.product(numpy.linspace(eq.lon-2., eq.lon+2., 20),
                                                numpy.linspace(eq.lat-2., eq.lat+2., 20))])
#
# for rw in lons_lats:
#     print(rw)
#
local_intensities = eq.local_intensities(ts=ts, lons=lons_lats[:,0], lats=lons_lats[:,1])

In [17]:
print(local_intensities.shape)

(100, 400)


In [ ]:
from globalETAS import Earthquake, deg2kmn
#
class NETAS_block(Earthquake):
    # Note: it might make more sense to include an Earthquke(), or list of Earthquake()s rather than subclassing.
    #. For batched MPP, for example, it would be possible to pre-construct a complete array of sites for multiple
    #  inputs (Earthquake()s). On the other hand, that introduces some opportunity for memory mis-management, 
    #. mistakes, or requiring indexing (aka, you have earthquakes that are not close together, maybe non-
    #. contiguous lattice sites)...
    def __init__(self, times=None, sptial_intensity_threshold=1e3, time_intensity_threshold=None, 
                 lon_0=0., lat_0=0., d_lon=.1, d_lat=.1,
                 *args, **kwargs):
        '''
        # @times: an iterable of times
        # @intensity_threshold: compute range by invertng spatial-omori; solve for 1/this.
        # @time_intensity_threshold: same, but for temporal distributionza
        # @lon_0, lat_0 : zero-point for lon, lat bins, respectively.
        # # parent array binning:
        #
        # TODO: what is the best parent binning convention? allow negative bins indices? Maybe we just
        #. leave that up to the parent object. I think that, by itself, negative index labels, centered
        #. on LL=[0,0] is most intuitive.
        # @d_lon, d_lat : lon, lat bin sizes, respectively
        #. bin indices are then j = int((lon - lon_0)/d_lon), k = int( (lat - lat_0)/d_lat)
        # 
        # parent __init__(): __init__(self, dict_or_recarray, transform_type='equal_area', transform_ratio_max=5.,
            ab_ratio_expon=.5, t_1=None, t_2=None)
        #
        '''
        #
        super(NETAS_block, self).__init__(*args, **kwargs)
        #
        # now, compute the spatial range and optionally the temporal range, or more specifically
        #   the lon, lat, time dimensions/indices. compute these as pairs: [[k_label, value], ...]
        #.  k_label: the index "label", or the index in a larger (global) lattice.
        #.  an we use their ordering for direct indexing.
        delta_lat = self.etas_range()/deg2km
        delta_lon = delta_lat*numpy.cos(self.lat*scipy.constants.deg)
        #
        lat_min = int((self.lat - delta_lat)/d_lat)*d_lat
        lon_min = int((self.lon - delta_lon)/d_lon)*d_lon
        #
        self.__dict__.update({'delta_lat':delta_lat, 'delta_lon':delta_lon, 'lat_min':lat_min, 'lon_min':lon_min})
        #
        # TODO: write these as functions to save memory? They will typically be called once
        #. and then return a 2D array of intensities.
        #lats = numpy.arange(lat_min, lat_min + 2*delta_lat, d_lat)
        #lats_index_labels = ((lats-lat_0)/d_lat).astype(int)
        #
        #lons = numpy.arange(lon_min, lon_min + 2*delta_lon, d_lon)
        #lons_index_labels = ((lons-lon_0)/d_lon).astype(int)
    #
    @property
    def lats(self):
        # TODO: are we de-modularizing too much here? I think maybe we just compute lats and lons
        #. at this level. we leave the binning to the calling function. 
        #lats = numpy.arange(self.lat_min, self.lat_min + 2*self.delta_lat, self.d_lat)
        #lats_index_labels = ((self.lats-self.lat_0)/self.d_lat).astype(int)
        #
        #return numpy.core.records.fromarrays([lats_index_labels, lats], dtype=[('index', '>i8'), ('lat', '>f8')])
        return numpy.arange(self.lat_min, self.lat_min + 2*self.delta_lat, self.d_lat)
                                    
    @property
    def lons(self):
        #lons = numpy.arange(self.lon_min, self.lon_min + 2*self.delta_lon, self.d_lon)
        #lons_index_labels = ((self.lons - self.lon_0)/self.d_lon).astype(int)
        #
        #return numpy.core.records.fromarrays([lons_index_labels, lons], dtype=[('index', '>i8'), ('lat', '>f8')])
        return numpy.arange(self.lon_min, self.lon_min + 2*self.delta_lon, self.d_lon)
    #
    def etas_range(self, spatial_intensity_threshold=None):
        spatial_intensity_threshold = spatial_intensity_threshold or self.spatial_intensity_threshold
        if spatial_intensity_threshold is None:
            return None
        #
        return self.r_0*(spatial_intensity_threshold**(1./self.q) - 1.)
    #
    def etas_temporal_range(self, time_intensity_threshold=None):
        time_intensity_threshold = time_intensity_threshold or self.time_intensity_threshold
        if time_intensity_threshold is None:
            return None
        #
        return self.t_0*(time_intensity_threshold**(1./self.p) - 1.)
                



In [19]:
print('** ', scipy.constants.degree*360.)

**  6.283185307179586


In [25]:
A = numpy.random.random(20)*20
print(A)
print(A.astype(int) )

[17.41622246  1.12833884  0.55006882  2.2878234  11.55561806 10.61805146
  7.55655403  0.30058825 11.84921565 11.08422654 18.09237713 19.52212661
  7.52433627 14.53836665 18.28883357 10.15502936  8.66872371  9.05863425
 18.3804989   8.78981498]
[17  1  0  2 11 10  7  0 11 11 18 19  7 14 18 10  8  9 18  8]


In [26]:
print(numpy.arange(5,10,.1))

[5.  5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6.  6.1 6.2 6.3 6.4 6.5 6.6 6.7
 6.8 6.9 7.  7.1 7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9 8.  8.1 8.2 8.3 8.4 8.5
 8.6 8.7 8.8 8.9 9.  9.1 9.2 9.3 9.4 9.5 9.6 9.7 9.8 9.9]


In [32]:
def f_lon(x):
    return (x+180)%360. - 180

for x in [-190., -120., -115., 120., 115., 190.]:
    print('** {} : {}'.format(x, f_lon(x)))
#


** -190.0 : 170.0
** -120.0 : -120.0
** -115.0 : -115.0
** 120.0 : 120.0
** 115.0 : 115.0
** 190.0 : -170.0
